In [ ]:
# pip install langgraph langchain_community langchain_core neo4j 
# pip install -U langchain-ollama langchain-neo4j langchain neo4j-graphrag
# pip install pyqlib

#### Alpha105(with yahoo)

##### 存入資料CSV取資料法(存完後可使用多商品調用)

In [57]:
import os
import pandas as pd
import yfinance as yf

qlib_data_dir = "../qlib_data/data"
os.makedirs(qlib_data_dir, exist_ok=True)

def get_stock_data_yfinance(ticker: str, period: str = "1y") -> pd.DataFrame:
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    df.reset_index(inplace=True)
    
    # 欄位名稱全部轉為小寫，以符合 Qlib 規範
    df.rename(columns={
        'Date': 'date',
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume'
    }, inplace=True)
    
    # 將 Volume 轉為 float
    df['volume'] = df['volume'].astype(float)
    
    # 將 datetime 轉為標準日期字串格式
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    
    df['symbol'] = ticker
    df['factor'] = 1.0
    df['k_high_low'] = df['close'] - df['open']

    df = df[['date', 'open', 'high', 'low', 'close', 'volume', 'factor', 'symbol', 'k_high_low']]
    return df

tickers = ["AAPL", "MSFT"]

for ticker in tickers:
    # 使用修正後的函數
    df = get_stock_data_yfinance(ticker, period="1y") 
    
    # 檢查 DataFrame 是否為空
    if df.empty:
        print(f"警告：股票 {ticker} 下載的資料為空，已跳過。")
        continue
    
    csv_path = os.path.join(qlib_data_dir, f"{ticker.upper()}.csv")
    df.to_csv(csv_path, index=False)
    print(f"股票 {ticker} 資料已存到 {csv_path}")

股票 AAPL 資料已存到 ../qlib_data/data/AAPL.csv
股票 MSFT 資料已存到 ../qlib_data/data/MSFT.csv


##### 存入資料CSV取資料法(存完後可使用單商品調用)

In [ ]:
import os
import pandas as pd
import yfinance as yf

# -------------------------------
# 1️⃣ 設定 QLib 本地資料夾 (無需變更)
# -------------------------------
qlib_data_dir = "../qlib_data"
features_day_dir = os.path.join(qlib_data_dir, "features", "day")
calendars_dir = os.path.join(qlib_data_dir, "calendars")
instruments_dir = os.path.join(qlib_data_dir, "instruments")

os.makedirs(features_day_dir, exist_ok=True)
os.makedirs(calendars_dir, exist_ok=True)
os.makedirs(instruments_dir, exist_ok=True)

# -------------------------------
# 2️⃣ 定義函數：從 yfinance 取得股票資料 (已修正)
# -------------------------------
def get_stock_data_yfinance(ticker: str, period: str = "1y") -> pd.DataFrame:
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    df.reset_index(inplace=True)
    
    # 欄位名稱全部轉為小寫，以符合 Qlib 規範
    df.rename(columns={
        'Date': 'date',
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume'
    }, inplace=True)
    
    # 將 Volume 轉為 float
    df['volume'] = df['volume'].astype(float)
    
    # 將 datetime 轉為標準日期字串格式
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    
    df['symbol'] = ticker
    df['factor'] = 1.0

    df = df[['date', 'open', 'high', 'low', 'close', 'volume', 'factor', 'symbol']]
    return df

# -------------------------------
# 3️⃣ 下載多支股票資料 (已修正)
# -------------------------------
tickers = ["AAPL", "MSFT"]
all_dates = set()

for ticker in tickers:
    # 使用修正後的函數
    df = get_stock_data_yfinance(ticker, period="1y") 
    
    # 檢查 DataFrame 是否為空
    if df.empty:
        print(f"警告：股票 {ticker} 下載的資料為空，已跳過。")
        continue

    all_dates.update(pd.to_datetime(df['date']).dt.date)
    
    csv_path = os.path.join(features_day_dir, f"{ticker.upper()}.csv")
    df.to_csv(csv_path, index=False)
    print(f"股票 {ticker} 資料已存到 {csv_path}")

# -------------------------------
# 4️⃣ 生成交易日曆 calendars/day.txt (無需變更)
# -------------------------------
if all_dates: # 確保下載到數據後才生成日曆
    calendar_path = os.path.join(calendars_dir, "day.txt")
    calendar_df = pd.DataFrame({"datetime": sorted(list(all_dates))})
    calendar_df['datetime'] = calendar_df['datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))
    calendar_df['datetime'].to_csv(calendar_path, index=False, header=False)
    print(f"交易日曆已生成: {calendar_path}")
else:
    print("錯誤：未能獲取任何交易數據，無法生成日曆。")


# -------------------------------
# 5️⃣ 生成股票列表 instruments/all.txt (已修正為更簡潔的格式)
# -------------------------------
if tickers:
    # 【修正 3】: 使用最簡單、最穩定的格式，每行只有一個股票代碼
    instruments_path = os.path.join(instruments_dir, "all.txt")
    with open(instruments_path, "w") as f:
        for ticker in tickers:
            f.write(f"{ticker.upper()}\n")
    print(f"股票列表已生成: {instruments_path}")

股票 AAPL 資料已存到 ./qlib_data/features/day/AAPL.csv
股票 MSFT 資料已存到 ./qlib_data/features/day/MSFT.csv
交易日曆已生成: ./qlib_data/calendars/day.txt
股票列表已生成: ./qlib_data/instruments/all.txt


In [ ]:
import pandas as pd
from qlib.data.dataset.loader import DataLoader

class CSVLoader(DataLoader):
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.df = None

    def load(self, instruments=None, start_time=None, end_time=None, fields=None):
        if self.df is None:
            df = pd.read_csv(self.csv_file, parse_dates=['date'])
            df = df.set_index(['date', 'symbol'])
            df.index.names = ['date', 'symbol']
            
            self.df = df
            df['label'] = df.groupby('symbol')['close'].pct_change().shift(-1)

        df = self.df

        if instruments is not None:
            df = df.loc[df.index.get_level_values('symbol').isin(instruments)]

        if start_time is not None:
            df = df[df.index.get_level_values('date') >= pd.to_datetime(start_time)]
        if end_time is not None:
            df = df[df.index.get_level_values('date') <= pd.to_datetime(end_time)]

        if fields is not None:
            df = df[fields]

        return df

    def fetch(self, instruments=None, start_time=None, end_time=None, fields=None):
        return self.load(instruments, start_time, end_time, fields)


from qlib.config import REG_US
import qlib
from qlib.data.dataset.handler import DataHandler

qlib.init(provider_uri="./qlib_data", region=REG_US)

id = "MSFT"
csv_file = f"./qlib_data/features/day/{id}.csv"
loader = CSVLoader(csv_file)
handler = DataHandler(
    instruments=[id],
    start_time="2024-09-20",
    end_time="2025-09-19",
    data_loader=loader,
)

# fetch 時直接取全部欄位
data = handler.fetch()
print(data.head())


##### 查詢當前套件底下的檔案名稱(取function用）

In [43]:
# ----------------------------------------
# 診斷步驟 2: 在 qlib 原始碼中搜尋 LocalProvider (無需變更)
# ----------------------------------------
import qlib
import os

try:
    extension = 'CSVHandler'
    # 獲取 qlib 的安裝根目錄
    qlib_path = qlib.__path__[0]
    print(f"開始在 qlib 模組安裝目錄中搜尋: {qlib_path}\n")

    found = False
    # 遍歷所有資料夾和檔案
    for root, dirs, files in os.walk(qlib_path):
        for file in files:
            # 只檢查 python 檔案
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        # 如果檔案內容包含 'class LocalProvider'
                        if f'class {extension}' in f.read():
                            print(f"*** 找到了！檔案位置在: ***\n{file_path}\n")
                            
                            # 從檔案路徑推導 import 路徑
                            relative_path = os.path.relpath(file_path, qlib_path)
                            import_path = "qlib." + relative_path.replace(os.sep, '.')[:-3] # 移除 .py
                            print(f"*** 請嘗試使用以下 import 指令: ***\nfrom {import_path} import {extension}")
                            found = True
                            break # 找到就不用再找了
                except Exception:
                    continue # 忽略讀取錯誤的檔案
        if found:
            break

    if not found:
        print(f"搜尋完畢，但在 qlib 模組中沒有找到 'class {extension}'。")
        print("這表示 pyqlib 安裝不完整或已損毀。")

except ImportError:
    print("錯誤：無法 'import qlib'。")
    print("請確認第一步中的 Python 直譯器路徑和 pyqlib 安裝位置是否匹配。如果不匹配，表示你的執行環境設定有誤。")
except Exception as e:
    print(f"執行搜尋時發生錯誤: {e}")

開始在 qlib 模組安裝目錄中搜尋: /home/hv/anaconda3/envs/statarb/lib/python3.11/site-packages/qlib

搜尋完畢，但在 qlib 模組中沒有找到 'class CSVHandler'。
這表示 pyqlib 安裝不完整或已損毀。


##### StaticLoader

In [45]:
# 使用StaticDataLoader导入数据
from qlib.data.dataset.loader import StaticDataLoader

# 实例化StaticDataLoader，主要是config函数，这里直接传入数据文件的路径
sdl_pkl = StaticDataLoader(config={'feature': '../qlib_data/data/AAPL.csv'})
sdl_pkl.load()['feature']  # 默认返回全部数据

/home/hv/anaconda3/envs/statarb/lib/python3.11/site-packages/qlib/utils/__init__.py:901: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(path_or_obj, parse_dates=True, index_col=index_col)


,,high,low,close,volume,factor,symbol
date,open,,,,,,
2024-09-23,226.286863,228.387090,224.763952,225.420898,54146000.0,1.0,AAPL
2024-09-24,227.590781,228.287550,224.684309,226.316711,43556100.0,1.0,AAPL
2024-09-25,223.888003,226.237071,222.982230,225.321335,42308700.0,1.0,AAPL
2024-09-26,226.247037,227.441475,224.365793,226.466019,36636700.0,1.0,AAPL
2024-09-27,227.401681,228.456769,226.247051,226.734772,34026000.0,1.0,AAPL
...,...,...,...,...,...,...,...
2025-09-16,237.179993,241.220001,236.320007,238.149994,63421100.0,1.0,AAPL
2025-09-17,238.970001,240.100006,237.729996,238.990005,46508000.0,1.0,AAPL
2025-09-18,239.970001,241.199997,236.649994,237.880005,44249600.0,1.0,AAPL


##### BIN

In [60]:
import qlib
from qlib.data import D
from qlib.contrib.data.handler import Alpha158
from qlib.data.dataset.processor import Fillna

# 初始化 Qlib
qlib.init(provider_uri="../qlib_data/my_data")

class CustomAlpha158(Alpha158):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print("CustomAlpha initialized")

    def get_feature_config(self):
        # 先拿到父類別定義的 feature
        expressions, names = super().get_feature_config()

        # 加上自定義 feature
        expressions += ["$k_high_low"]
        names += ["custom_ratio"]

        print("Custom features added:", names[-1])
        return expressions, names

# 獲取數據
instruments = ["msft", "aapl"]
fields = ["$open", "$close", "$high", "$low", "$volume", "$factor", '$k_high_low']
start_time = "2024-09-20"
end_time = "2025-09-22"

# 獲取 OHLCV 數據
data = D.features(instruments=instruments, fields=fields, start_time=start_time, end_time=end_time)
print("Input data columns:", data.columns)
print("Input data head:", data.head())

# 將欄位名稱轉為小寫
data.columns = [col.lower() for col in data.columns]

# 配置處理器
data_handler = CustomAlpha158(
    instruments=instruments,
    start_time=start_time,
    end_time=end_time,
    freq="day",
    infer_processors=[Fillna(fields_group="feature")],
    learn_processors=[],
)

# 計算因子
factor_data = data_handler.fetch()
print("Factor data columns:", factor_data.columns)
print("Factor data head:", factor_data.head())

# 檢查 AAPL 的 custom_ratio
appl_data1 = factor_data.xs("aapl", level="instrument")

[7096:MainThread](2025-09-23 17:36:06,924) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[7096:MainThread](2025-09-23 17:36:06,925) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[7096:MainThread](2025-09-23 17:36:06,925) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/home/hv/Desktop/quant/backtest/qlib_data/my_data')}


Input data columns: Index(['$open', '$close', '$high', '$low', '$volume', '$factor',
       '$k_high_low'],
      dtype='object')
Input data head:                             $open      $close       $high        $low  \
instrument datetime                                                     
aapl       2024-09-23  226.286865  225.420898  228.387085  224.763947   
           2024-09-24  227.590775  226.316711  228.287552  224.684311   
           2024-09-25  223.888000  225.321335  226.237076  222.982224   
           2024-09-26  226.247040  226.466019  227.441467  224.365799   
           2024-09-27  227.401688  226.734772  228.456772  226.247055   

                          $volume  $factor  $k_high_low  
instrument datetime                                      
aapl       2024-09-23  54146000.0      1.0    -0.865965  
           2024-09-24  43556100.0      1.0    -1.274069  
           2024-09-25  42308700.0      1.0     1.433332  
           2024-09-26  36636700.0      1.0     0.21

[7096:MainThread](2025-09-23 17:36:07,386) INFO - qlib.timer - [log.py:127] - Time cost: 0.322s | Loading data Done
[7096:MainThread](2025-09-23 17:36:07,397) INFO - qlib.timer - [log.py:127] - Time cost: 0.010s | Fillna Done
[7096:MainThread](2025-09-23 17:36:07,398) INFO - qlib.timer - [log.py:127] - Time cost: 0.012s | fit & process data Done
[7096:MainThread](2025-09-23 17:36:07,398) INFO - qlib.timer - [log.py:127] - Time cost: 0.335s | Init data Done


CustomAlpha initialized
Factor data columns: Index(['KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT',
       'KSFT2', 'OPEN0',
       ...
       'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20',
       'VSUMD30', 'VSUMD60', 'custom_ratio', 'LABEL0'],
      dtype='object', length=160)
Factor data head:                            KMID      KLEN     KMID2       KUP      KUP2  \
datetime   instrument                                                     
2024-09-23 aapl       -0.003827  0.016011 -0.239010  0.009281  0.579669   
           msft       -0.001773  0.013977 -0.126852  0.005020  0.359145   
2024-09-24 aapl       -0.005598  0.015832 -0.353588  0.003062  0.193375   
           msft       -0.008845  0.016744 -0.528276  0.000808  0.048275   
2024-09-25 aapl        0.006402  0.014538  0.440368  0.004090  0.281346   

                           KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...  \
datetime   instrument                                        

In [61]:
print("AAPL data columns:", appl_data1.columns)
print("AAPL custom_ratio tail:", appl_data1[['custom_ratio']].tail())

AAPL data columns: Index(['KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT',
       'KSFT2', 'OPEN0',
       ...
       'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20',
       'VSUMD30', 'VSUMD60', 'custom_ratio', 'LABEL0'],
      dtype='object', length=160)
AAPL custom_ratio tail:             custom_ratio
datetime                
2025-09-16      0.970001
2025-09-17      0.020004
2025-09-18     -2.089996
2025-09-19      4.270004
2025-09-22      7.779984


In [55]:
import qlib
from qlib.data import D
from qlib.contrib.data.handler import Alpha360
from qlib.data.dataset.processor import Fillna

# 初始化 Qlib
qlib.init(provider_uri="../qlib_data/my_data")

# 獲取數據
instruments = ["msft", "aapl"]
fields = ["$open", "$close", "$high", "$low", "$volume", "$factor"]
start_time = "2024-09-20"
end_time = "2025-09-22"

# 檢查原始數據
data = D.features(instruments=instruments, fields=fields, start_time=start_time, end_time=end_time)
print("Input data columns:", data.columns)
print("Input data head:", data.head())
print("Missing values in input data:", data[['$open', '$close']].isna().sum())

# 將欄位名稱轉為小寫
data.columns = [col.lower() for col in data.columns]

# 配置處理器
data_handler = Alpha360(
    instruments=instruments,
    start_time=start_time,
    end_time=end_time,
    freq="day",
    infer_processors=[Fillna()],  # 移除 fields_group
    learn_processors=[],
)

# 計算因子
factor_data = data_handler.fetch()
print("Factor data columns:", factor_data.columns.tolist())
print("Factor data head:", factor_data.tail())
print("Factor data index levels:", factor_data.index.names)

appl_data2 = factor_data.xs("aapl", level="instrument")

[7096:MainThread](2025-09-23 17:29:13,064) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[7096:MainThread](2025-09-23 17:29:13,065) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[7096:MainThread](2025-09-23 17:29:13,065) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/home/hv/Desktop/quant/backtest/qlib_data/my_data')}


Input data columns: Index(['$open', '$close', '$high', '$low', '$volume', '$factor'], dtype='object')
Input data head:                             $open      $close       $high        $low  \
instrument datetime                                                     
aapl       2024-09-23  226.286865  225.420898  228.387085  224.763947   
           2024-09-24  227.590775  226.316711  228.287552  224.684311   
           2024-09-25  223.888000  225.321335  226.237076  222.982224   
           2024-09-26  226.247040  226.466019  227.441467  224.365799   
           2024-09-27  227.401688  226.734772  228.456772  226.247055   

                          $volume  $factor  
instrument datetime                         
aapl       2024-09-23  54146000.0      1.0  
           2024-09-24  43556100.0      1.0  
           2024-09-25  42308700.0      1.0  
           2024-09-26  36636700.0      1.0  
           2024-09-27  34026000.0      1.0  
Missing values in input data: $open     0
$close    0


[7096:MainThread](2025-09-23 17:29:13,555) INFO - qlib.timer - [log.py:127] - Time cost: 0.339s | Loading data Done
[7096:MainThread](2025-09-23 17:29:13,557) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | Fillna Done
[7096:MainThread](2025-09-23 17:29:13,557) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | fit & process data Done
[7096:MainThread](2025-09-23 17:29:13,558) INFO - qlib.timer - [log.py:127] - Time cost: 0.342s | Init data Done


Factor data columns: ['CLOSE59', 'CLOSE58', 'CLOSE57', 'CLOSE56', 'CLOSE55', 'CLOSE54', 'CLOSE53', 'CLOSE52', 'CLOSE51', 'CLOSE50', 'CLOSE49', 'CLOSE48', 'CLOSE47', 'CLOSE46', 'CLOSE45', 'CLOSE44', 'CLOSE43', 'CLOSE42', 'CLOSE41', 'CLOSE40', 'CLOSE39', 'CLOSE38', 'CLOSE37', 'CLOSE36', 'CLOSE35', 'CLOSE34', 'CLOSE33', 'CLOSE32', 'CLOSE31', 'CLOSE30', 'CLOSE29', 'CLOSE28', 'CLOSE27', 'CLOSE26', 'CLOSE25', 'CLOSE24', 'CLOSE23', 'CLOSE22', 'CLOSE21', 'CLOSE20', 'CLOSE19', 'CLOSE18', 'CLOSE17', 'CLOSE16', 'CLOSE15', 'CLOSE14', 'CLOSE13', 'CLOSE12', 'CLOSE11', 'CLOSE10', 'CLOSE9', 'CLOSE8', 'CLOSE7', 'CLOSE6', 'CLOSE5', 'CLOSE4', 'CLOSE3', 'CLOSE2', 'CLOSE1', 'CLOSE0', 'OPEN59', 'OPEN58', 'OPEN57', 'OPEN56', 'OPEN55', 'OPEN54', 'OPEN53', 'OPEN52', 'OPEN51', 'OPEN50', 'OPEN49', 'OPEN48', 'OPEN47', 'OPEN46', 'OPEN45', 'OPEN44', 'OPEN43', 'OPEN42', 'OPEN41', 'OPEN40', 'OPEN39', 'OPEN38', 'OPEN37', 'OPEN36', 'OPEN35', 'OPEN34', 'OPEN33', 'OPEN32', 'OPEN31', 'OPEN30', 'OPEN29', 'OPEN28', 'OPEN27'

##### 單商品調用

In [1]:
import qlib
from qlib.data import D
from qlib.contrib.data.handler import Alpha158
from qlib.data.dataset.processor import RobustZScoreNorm, Fillna, DropnaProcessor, CSRankNorm

# 初始化 Qlib
qlib.init(provider_uri="../qlib_data/my_data")

# 獲取數據
instruments = ["MSFT"]
fields = ["$open", "$close", "$high", "$low", "$volume"]
start_time = "2024-09-20"
end_time = "2025-09-22"

# 獲取 OHLCV 數據
data = D.features(instruments=instruments, fields=fields, start_time=start_time, end_time=end_time)

# 將欄位名稱轉為小寫，以符合 Alpha158 的要求
data.columns = [col.lower() for col in data.columns]

print("Fetched data head:")
print(data.head())

# 配置 Alpha158 的處理器
infer_processors = [
    RobustZScoreNorm(fields_group="feature", fit_start_time=start_time, fit_end_time=end_time),  # 標準化
    Fillna(fields_group="feature"),  # 填補缺失值
]
learn_processors = [
    DropnaProcessor(),  # 移除缺失值
    CSRankNorm(fields_group="feature"),  # 截面排名正規化
]

# 初始化 Alpha158 數據處理器
data_handler = Alpha158(
    instruments=instruments,
    start_time=start_time,
    end_time=end_time,
    freq="day",  # 數據頻率，根據你的數據設定
    infer_processors=infer_processors,
    learn_processors=learn_processors,
)

# 計算因子
factors = data_handler.get_cols()

# 獲取因子數據
factor_data = data_handler.fetch()

print("Calculated Alpha158 factors head:")
print(factor_data.head())

[1076106:MainThread](2025-09-23 12:10:48,537) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[1076106:MainThread](2025-09-23 12:10:48,724) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[1076106:MainThread](2025-09-23 12:10:48,725) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/home/hv/Desktop/quant/backtest/qlib_data/my_data')}
[1076106:MainThread](2025-09-23 12:10:48,768) INFO - qlib.timer - [log.py:127] - Time cost: 0.037s | Loading data Done
/home/hv/anaconda3/envs/statarb/lib/python3.11/site-packages/qlib/data/dataset/processor.py:285: RuntimeWarning: All-NaN slice encountered
  self.mean_train = np.nanmedian(X, axis=0)
/home/hv/anaconda3/envs/statarb/lib/python3.11/site-packages/qlib/data/dataset/processor.py:286: RuntimeWarning: All-NaN slice encountered
  self.std_train = np.nanmedian(np.abs(X - self.mean_train), axis=0)
[1076106:MainThread](2025-09-23 12:10:

Fetched data head:
                            $open      $close       $high        $low  \
instrument datetime                                                     
MSFT       2024-09-20  433.961975  432.026489  435.966919  430.984314   
           2024-09-23  431.043915  430.279663  433.207672  427.182922   
           2024-09-24  429.773468  425.972015  430.120850  422.924896   
           2024-09-25  426.627075  428.890076  429.892548  425.376465   
           2024-09-26  431.847870  428.096039  432.056305  425.932281   

                          $volume  
instrument datetime                
MSFT       2024-09-20  55167100.0  
           2024-09-23  15128900.0  
           2024-09-24  17015800.0  
           2024-09-25  13396400.0  
           2024-09-26  14492000.0  
Calculated Alpha158 factors head:
                           KMID      KLEN     KMID2       KUP      KUP2  \
datetime   instrument                                                     
2024-09-20 MSFT       -0.537441 -

#### LangGraph

In [ ]:
import os, json, time
import requests
from typing import TypedDict, Literal
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from IPython.display import Image, display

# --- 輔助函式：手動卸載模型 ---
def unload_ollama_model(model_name: str, base_url: str = "http://localhost:11434"):  # 注意：Ollama 預設端口為 11434，而非 11435
    """
    手動呼叫 Ollama API 來卸載指定的模型。
    通過發送一個空提示的 generate 請求，並設定 keep_alive=0 來實現卸載。
    """
    print(f"\n--- 正在請求從 GPU 卸載模型: {model_name} ---")
    try:
        url = f"{base_url}/api/generate"
        payload = {
            "model": model_name,
            "prompt": "",  # 空提示，避免不必要的生成
            "keep_alive": 0  # 設定 keep_alive=0 以立即卸載，注意：移到頂層，而非 options 中
        }
        response = requests.post(url, json=payload)
        response.raise_for_status()
        print(f"✅ 成功發送卸載請求。")
    except requests.exceptions.RequestException as e:
        print(f"❌ 卸載模型時發生錯誤: {e}")
        print("請確認 Ollama 服務正在運行，且 base_url 設定正確（預設為 http://localhost:11434）。")

# --- 模型名稱與位址設定 ---
MODEL_NAME = "mixtral:8x7b"
OLLAMA_BASE_URL = "http://localhost:11435" 

# --- 模型初始化 ---
classifier_llm = Ollama(base_url=OLLAMA_BASE_URL, model=MODEL_NAME, temperature=0)
generator_llm = Ollama(base_url=OLLAMA_BASE_URL, model=MODEL_NAME, temperature=0.1)

# --- 1. 定義狀態 (State) ---
class GraphState(TypedDict):
    topic: str  # 輸入主題，例如股票代碼或金融查詢
    alpha105_features: str | None  # Alpha105 提取的特徵
    alpha360_features: str | None  # Alpha360 提取的特徵
    alpha101_features: str | None  # WorldQuant Alpha101 提取的特徵
    integrated_answer: str  # 最終整合答案

# --- 2. 定義節點 (Nodes) ---
def alpha105_node(state: GraphState):
    print("--- 節點: alpha105_node ---")
    topic = state['topic']
    print(f"正在為 '{topic}' 提取 Alpha105 特徵...")
    # 模擬提取：實際中可使用代碼計算因子（如使用 numpy/pandas 處理 OHLCV 資料）
    features = f"Alpha105 特徵提取結果 for {topic}: [模擬特徵1: 0.85, 模擬特徵2: -0.12, ...]（假設基於歷史價格、成交量計算的105個因子）"
    return {"alpha105_features": features}

def alpha360_node(state: GraphState):
    print("--- 節點: alpha360_node ---")
    topic = state['topic']
    print(f"正在為 '{topic}' 提取 Alpha360 特徵...")
    # 模擬提取：實際中可整合更多資料源計算360個因子
    features = f"Alpha360 特徵提取結果 for {topic}: [模擬特徵1: 1.23, 模擬特徵2: 0.45, ...]（基於多維度金融資料的360個因子）"
    return {"alpha360_features": features}

def worldquant_alpha101_node(state: GraphState):
    print("--- 節點: worldquant_alpha101_node ---")
    topic = state['topic']
    print(f"正在為 '{topic}' 提取 WorldQuant Alpha101 特徵...")
    # 模擬提取：WorldQuant 的 101 個 alpha 因子，基於股票資料計算
    features = f"WorldQuant Alpha101 特徵提取結果 for {topic}: [模擬特徵1: 0.67, 模擬特徵2: -0.34, ...]（101個經典因子，如 rank(correlation(volume, high))）"
    return {"alpha101_features": features}

def integrate_features_node(state: GraphState):
    print("--- 節點: integrate_features_node ---")
    topic = state['topic']
    alpha105 = state['alpha105_features']
    alpha360 = state['alpha360_features']
    alpha101 = state['alpha101_features']
    
    prompt = ChatPromptTemplate.from_template(
    """
        你是一個金融分析專家。請根據以下主題和從 Alpha105、Alpha360、WorldQuant Alpha101 提取的特徵，用繁體中文生成一份詳細的綜合分析報告。整合所有特徵，提供投資建議或洞見。
        主題: {topic}
        Alpha105 特徵: {alpha105}
        Alpha360 特徵: {alpha360}
        WorldQuant Alpha101 特徵: {alpha101}

        綜合分析:
    """
    )
    chain = prompt | generator_llm
    integrated_answer = chain.invoke({
        "topic": topic,
        "alpha105": alpha105 or "無資料",
        "alpha360": alpha360 or "無資料",
        "alpha101": alpha101 or "無資料"
    }).strip()
    
    if not integrated_answer:
        integrated_answer = f"無法生成綜合分析，請檢查特徵提取或模型配置。"
    print(f"生成的綜合分析: {integrated_answer}")
    return {"integrated_answer": integrated_answer}

# --- 3. 定義圖的建構 ---
# 獨立工作流：從入點開始，順序執行三個提取節點（可視為獨立計算方式），然後整合
# 如果需要平行，可考慮使用 langgraph 的分支，但這裡使用簡單順序以確保狀態更新
workflow = StateGraph(GraphState)
workflow.add_node("alpha105_node", alpha105_node)
workflow.add_node("alpha360_node", alpha360_node)
workflow.add_node("worldquant_alpha101_node", worldquant_alpha101_node)
workflow.add_node("integrate_features_node", integrate_features_node)

# 設定入點並連接邊：順序執行提取，然後整合
workflow.set_entry_point("alpha105_node")
workflow.add_edge("alpha105_node", "alpha360_node")
workflow.add_edge("alpha360_node", "worldquant_alpha101_node")
workflow.add_edge("worldquant_alpha101_node", "integrate_features_node")
workflow.add_edge("integrate_features_node", END)

app = workflow.compile()

# --- 4. 執行服務與檢查 ---
if __name__ == '__main__':
    inputs = {"topic": "AAPL 股票分析"}

    try:
        print("--- 🚀 開始執行 LangGraph 服務 ---")
        
        # 新增：測量運行時間 - 開始計時
        start_time = time.time()
        
        final_state = app.invoke(inputs)
        
        # 新增：測量運行時間 - 結束計時並輸出
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"\n--- LangGraph 整體運行時間: {execution_time:.2f} 秒 ---")
        
        print("\n--- ✅ 服務執行完畢 ---")
        print("--- 最終答案 ---")
        print(final_state['answer'])

    finally:
        # finally 區塊確保無論服務成功或失敗，這段程式碼都會被執行
        # 在生成最終答案後直接卸載模型
        unload_ollama_model(model_name=MODEL_NAME, base_url=OLLAMA_BASE_URL)

--- 🚀 開始執行 LangGraph 服務 ---
--- 節點: categorize_topic ---
⚠️ 模型返回無效分類: ，預設為 '簡單'
主題 '台灣最高的山是哪一座？' 被分類為: 簡單
--- 條件邊: decide_next_node ---
判斷結果: 簡單 -> 前往 simple_answer_node
--- 節點: simple_answer_node ---
生成的簡單答案: 台灣最高峰為「雪山」，海拔高達五千四十三公尺。

--- LangGraph 整體運行時間: 12.75 秒 ---

--- ✅ 服務執行完畢 ---
--- 最終答案 ---
台灣最高峰為「雪山」，海拔高達五千四十三公尺。

--- 正在請求從 GPU 卸載模型: mixtral:8x7b ---
✅ 成功發送卸載請求。


#### Qdrant

In [1]:
# 1. 導入必要的模組
import torch
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer

# --- 配置 ---
# Qdrant 伺服器的地址和端口
# 如果您的 Qdrant 是在 Docker 中或遠端機器上運行的，請將 "localhost" 更改為對應的 IP 地址
QDRANT_HOST = "localhost"
QDRANT_GRPC_PORT = 6333

# 我們要創建的集合 (Collection) 的名稱
COLLECTION_NAME = "common_sense_collection"

# 選擇一個嵌入模型。'distiluse-base-multilingual-cased-v1' 是一個支援多語言（包含繁體中文）的好選擇。
# 第一次運行時，程式會自動下載此模型，可能會需要一些時間。
EMBEDDING_MODEL_NAME = 'distiluse-base-multilingual-cased-v1'

# 2. 準備要添加的常識資料
# 您可以換成任何您想添加的文字資料
documents = [
    "天空是藍色的。",
    "草是綠色的。",
    "太陽會從東邊升起，西邊落下。",
    "大部分的鳥類都會飛。",
    "水在攝氏零度時會結冰。",
    "蘋果是一種可以吃的水果。",
    "一加一等於二。",
    "台灣的首都是台北。",
]

# 3. 初始化 Qdrant 客戶端和嵌入模型
# 連接到 Qdrant 伺服器
# qdrant-client 預設使用 gRPC (port 6633)，通常性能更好
client = QdrantClient(host=QDRANT_HOST, port=QDRANT_GRPC_PORT)


In [2]:

# 檢查是否有可用的 GPU，否則使用 CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用的計算設備: {device}")

# 加載嵌入模型
# 將模型加載到指定的設備 (GPU 或 CPU)
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=device)

# 獲取模型的向量維度，這在創建 Collection 時需要
# 這一步會運行一個假的句子來獲取輸出向量的大小
vector_size = embedding_model.get_sentence_embedding_dimension()
print(f"使用的模型 '{EMBEDDING_MODEL_NAME}' 的向量維度是: {vector_size}")

# 4. 創建 Qdrant 集合 (Collection)
# 使用 recreate_collection，如果同名的 Collection 已存在，會先刪除再重建。
# 這在測試時很方便，可以確保每次運行都是一個乾淨的狀態。
# 如果您只想在已有的 Collection 中添加資料，請確保這段代碼不會重複執行，或改用 client.get_collection 來檢查是否存在。
try:
    client.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=vector_size,          # 向量維度，必須與模型輸出一致
            distance=models.Distance.COSINE  # 距離函數，COSINE 對於文字向量通常效果很好
        )
    )
    print(f"集合 '{COLLECTION_NAME}' 創建成功！")
except Exception as e:
    print(f"創建集合時出錯: {e}")


# 5. 將文字資料轉換為向量並準備上傳
print("正在將文字轉換為向量...")
# 這一步會將所有 documents 轉換為向量，如果資料量大，會需要較長時間
vectors = embedding_model.encode(documents, show_progress_bar=True)

print("向量生成完畢，準備上傳資料...")
# 6. 上傳資料到 Qdrant (Upsert)
# 我們將每個資料點包裝成一個 PointStruct 對象
# 包含 id、vector 和 payload
# payload 可以存放原始文字或其他元數據 (metadata)
client.upsert(
    collection_name=COLLECTION_NAME,
    points=[
        models.PointStruct(
            id=idx,  # 每個點都需要一個唯一的 ID，這裡我們用簡單的索引
            vector=vector.tolist(),  # 向量
            payload={"text": doc}    # 附加資料，這裡我們把原始句子存起來
        )
        for idx, (doc, vector) in enumerate(zip(documents, vectors))
    ],
    wait=True  # 設置為 True，確保操作完成後再繼續
)

print("資料上傳成功！")

# --- 驗證 ---
# 讓我們來驗證一下資料是否真的上傳成功了
try:
    collection_info = client.get_collection(collection_name=COLLECTION_NAME)
    print(f"\n--- 驗證成功 ---")
    print(f"集合 '{COLLECTION_NAME}' 的資訊: ")
    print(f"  - 向量數量: {collection_info.points_count}")
    print("\n您可以前往 Qdrant 的 Web UI 查看更多細節:")
    print(f"http://{QDRANT_HOST}:6634/dashboard")
except Exception as e:
    print(f"\n--- 驗證失敗 ---")
    print(f"無法獲取集合 '{COLLECTION_NAME}' 的資訊: {e}")

使用的計算設備: cuda
使用的模型 'distiluse-base-multilingual-cased-v1' 的向量維度是: 512
集合 'common_sense_collection' 創建成功！
正在將文字轉換為向量...


/tmp/ipykernel_1783291/606891131.py:19: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

向量生成完畢，準備上傳資料...
資料上傳成功！

--- 驗證成功 ---
集合 'common_sense_collection' 的資訊: 
  - 向量數量: 8

您可以前往 Qdrant 的 Web UI 查看更多細節:
http://localhost:6634/dashboard


#### Neo4j

In [71]:
import yfinance as yf
import pandas as pd
import numpy as np

# 股票代碼
tickers = ["0050.TW", "2330.TW"]

# 取得近1年的日K資料
data = yf.download(tickers, period="1y", interval="1d")

# 將多層欄位轉單層欄位
data.columns = ['_'.join(col) for col in data.columns]

# 計算5日、10日移動平均 (註: 這是SMA, 若要真EMA可改用ewm.mean())
data['EMA5_0050'] = data['Close_0050.TW'].rolling(window=5).mean()
data['EMA10_0050'] = data['Close_0050.TW'].rolling(window=10).mean()
data['EMA5_2330'] = data['Close_2330.TW'].rolling(window=5).mean()
data['EMA10_2330'] = data['Close_2330.TW'].rolling(window=10).mean()

# 計算ATR（Average True Range）
def compute_atr(df, symbol, period=14):
    high = df[f'High_{symbol}']
    low = df[f'Low_{symbol}']
    close = df[f'Close_{symbol}']
    
    tr = pd.concat([
        high - low,
        (high - close.shift(1)).abs(),
        (low - close.shift(1)).abs()
    ], axis=1).max(axis=1)
    
    atr = tr.rolling(window=period).mean()  # 或使用EMA: tr.ewm(span=period, adjust=False).mean()
    return atr

data['ATR_0050'] = compute_atr(data, '0050.TW')
data['ATR_2330'] = compute_atr(data, '2330.TW')

# 計算報酬率（日報酬率）
data['Return_0050'] = data['Close_0050.TW'].pct_change(fill_method=None)
data['Return_2330'] = data['Close_2330.TW'].pct_change(fill_method=None)

# 計算收盤價差並插入/更新關係
data['price_diff'] = (data['Close_2330.TW'] - data['Close_0050.TW']) / (data['Close_2330.TW'] + data['Close_0050.TW'])

# (可選) 移除因diff()產生的額外NaN (第一行diff會是NaN)
data = data.dropna()

# 查看結果
print(data.head(5))


[*********************100%***********************]  2 of 2 completed

            Close_0050.TW  Close_2330.TW  High_0050.TW  High_2330.TW  \
Date                                                                   
2024-10-11      47.051067    1027.682129     47.210233   1032.599268   
2024-10-14      47.087799    1027.682129     47.234717   1037.516408   
2024-10-15      48.091747    1052.267700     48.140724   1057.184839   
2024-10-16      47.259205    1027.682129     47.822398   1052.267826   
2024-10-17      47.332664    1017.847717     47.528558   1037.516272   

            Low_0050.TW  Low_2330.TW  Open_0050.TW  Open_2330.TW  \
Date                                                               
2024-10-11    46.291984  1003.096432     46.291984   1008.013571   
2024-10-14    46.757228  1017.847850     47.038827   1027.682129   
2024-10-15    47.259203  1032.599145     47.259203   1032.599145   
2024-10-16    46.928634  1017.847850     47.124528   1022.764990   
2024-10-17    46.989849  1012.930579     47.504070   1032.599133   

            Volume

##### 插入

In [74]:
from neo4j import GraphDatabase

# Neo4j 連線設定
uri = "bolt://localhost:7687"
user = "neo4j"
password = "test1234"
driver = GraphDatabase.driver(uri, auth=(user, password))

def insert_stock_kline(tx, stock_name, date, kline_data):
    query_kline = """
    MERGE (t:Time {date:$date})
    MERGE (s:Stock {name:$stock_name})
    MERGE (k:Kline {date:$date, stock:$stock_name})
      ON CREATE SET k.open=$open, k.high=$high, k.low=$low, k.close=$close, k.volume=$volume
      ON MATCH SET k.open=$open, k.high=$high, k.low=$low, k.close=$close, k.volume=$volume
    MERGE (s)-[:STOCK_AT_TIME]->(t)
    MERGE (t)-[:HAS_KLINE]->(k)
    """
    tx.run(query_kline,
           stock_name=stock_name,
           date=date,
           open=kline_data['Open'],
           high=kline_data['High'],
           low=kline_data['Low'],
           close=kline_data['Close'],
           volume=kline_data['Volume'])

    # 動態插入技術指標（除了 OHLCV 以外的欄位）
    exclude = {"Open", "High", "Low", "Close", "Volume"}
    for indicator_name, value in kline_data.items():
        if indicator_name not in exclude:
            query_indicator = f"""
                MATCH (k:Kline {{date:$date, stock:$stock_name}})
                MERGE (ind:{indicator_name} {{stock:$stock_name, date:$date}})
                SET ind.value = $value
                MERGE (k)-[:HAS_{indicator_name.upper()}]->(ind)
            """
            tx.run(query_indicator,
                   stock_name=stock_name,
                   date=date,
                   indicator_name=indicator_name,
                   value=value)

# 插入每日收盤價差，會自動覆蓋
def insert_price_diff(tx, date, stock1, stock2, diff):
    query = """
    MERGE (pd:PriceDiff {date:$date, stock1:$stock1, stock2:$stock2})
    SET pd.value = $diff
    WITH pd
    MATCH (k1:Kline {date:$date, stock:$stock1})
    MATCH (k2:Kline {date:$date, stock:$stock2})
    MERGE (pd)-[:FROM_KLINE]->(k1)
    MERGE (pd)-[:FROM_KLINE]->(k2)
    """
    tx.run(query, date=date, stock1=stock1, stock2=stock2, diff=diff)

# 將資料寫入 Neo4j
def insert_data_to_neo4j(data):
    with driver.session() as session:
        for date, row in data.iterrows():
            # 插入 0050 K 線
            session.execute_write(insert_stock_kline, "0050.TW", str(date.date()), {
                'Open': row['Open_0050.TW'],
                'High': row['High_0050.TW'],
                'Low': row['Low_0050.TW'],
                'Close': row['Close_0050.TW'],
                'Volume': row['Volume_0050.TW'],
                'EMA5': row['EMA5_0050'],
                'EMA10': row['EMA10_0050'],
                'ATR': row['ATR_0050'],
                'Return': row['Return_0050']
            })
            # 插入 2330 K 線
            session.execute_write(insert_stock_kline, "2330.TW", str(date.date()), {
                'Open': row['Open_2330.TW'],
                'High': row['High_2330.TW'],
                'Low': row['Low_2330.TW'],
                'Close': row['Close_2330.TW'],
                'Volume': row['Volume_2330.TW'],
                'EMA5': row['EMA5_2330'],
                'EMA10': row['EMA10_2330'],
                'ATR': row['ATR_2330'],
                'Return': row['Return_2330']
            })
            
            # 計算收盤價差並插入/更新關係
            session.execute_write(insert_price_diff, str(date.date()), "2330.TW", "0050.TW", row['price_diff'])

if __name__ == "__main__":
    if not data.empty:
        print("開始將資料寫入 Neo4j...")
        insert_data_to_neo4j(data)
        print("資料寫入完成！")
    else:
        print("資料是空的，沒有執行寫入操作。")

    driver.close()


開始將資料寫入 Neo4j...
資料寫入完成！


##### 刪除

In [72]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
user = "neo4j"
password = "test1234"
driver = GraphDatabase.driver(uri, auth=(user, password))

def clear_neo4j(tx):
    # DETACH DELETE 會先刪掉節點的所有關係再刪節點
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session() as session:
    session.execute_write(clear_neo4j)
    print("已經清空 Neo4j 資料庫！")

driver.close()


已經清空 Neo4j 資料庫！


##### 查看

In [75]:
from neo4j import GraphDatabase

# --- 你的連線設定和插入函式放在這裡 ---
def check_data_in_neo4j(driver, sample_size=3):
    """查詢並印出節點總數與部分節點內容"""

    def get_node_counts(tx):
        result = tx.run("MATCH (n) RETURN labels(n) AS Label, count(n) AS Count")
        return [record.data() for record in result]

    def get_sample_nodes(tx, label, limit):
        query = f"MATCH (n:{label}) RETURN n LIMIT {limit}"
        result = tx.run(query)
        return [record["n"] for record in result]

    with driver.session() as session:
        counts = session.execute_read(get_node_counts)
        print("--- 資料庫節點統計 ---")
        if not counts:
            print("資料庫是空的！")
        else:
            for record in counts:
                label = record["Label"][0] if record["Label"] else "Unknown"
                count = record["Count"]
                print(f"標籤: {label}, 數量: {count}")
                # 顯示樣本節點
                samples = session.execute_read(get_sample_nodes, label, sample_size)
                for i, node in enumerate(samples, 1):
                    print(f"  範例 {i}: {dict(node)}")
        print("--------------------")

uri = "bolt://localhost:7687"
user = "neo4j"
password = "test1234"
driver = GraphDatabase.driver(uri, auth=(user, password))

check_data_in_neo4j(driver)

driver.close()

--- 資料庫節點統計 ---
標籤: Return, 數量: 436
  範例 1: {'date': '2024-10-14', 'stock': '2330.TW', 'value': 0.0}
  範例 2: {'date': '2024-10-15', 'stock': '0050.TW', 'value': 0.021320771653165593}
  範例 3: {'date': '2024-10-15', 'stock': '2330.TW', 'value': 0.023923322783892864}
標籤: PriceDiff, 數量: 218
  範例 1: {'date': '2024-10-14', 'stock1': '2330.TW', 'value': 0.9123760391010738, 'stock2': '0050.TW'}
  範例 2: {'date': '2024-10-15', 'stock1': '2330.TW', 'value': 0.9125890228068542, 'stock2': '0050.TW'}
  範例 3: {'date': '2024-10-16', 'stock1': '2330.TW', 'value': 0.9120710993618174, 'stock2': '0050.TW'}
標籤: Time, 數量: 218
  範例 1: {'date': '2024-10-15'}
  範例 2: {'date': '2024-10-16'}
  範例 3: {'date': '2024-10-17'}
標籤: Kline, 數量: 436
  範例 1: {'date': '2024-10-15', 'volume': 59070828.0, 'high': 48.140723540621785, 'low': 47.25920322936276, 'stock': '0050.TW', 'close': 48.09174728393555, 'open': 47.25920322936276}
  範例 2: {'date': '2024-10-15', 'volume': 42850176.0, 'high': 1057.184838981272, 'low': 1032.59

In [70]:
from langchain_community.llms import Ollama
from langchain_neo4j import Neo4jGraph
from langchain.prompts import PromptTemplate
from neo4j import GraphDatabase
import json
import pandas as pd

# --- 模型名稱與位址設定 ---
generator_llm = Ollama(base_url="http://localhost:11435", model="mixtral:8x7b", temperature=0.3)  # Increased temperature for better output

# --- Neo4j 連線設定 ---
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "test1234"

# 建立 Neo4jGraph 物件
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD
)

# 刷新 schema，確保 graph 物件有最新的資料庫結構資訊
graph.refresh_schema()

# 檢查 schema 是否正確
print("\nNeo4j Schema:")
print(graph.schema)

# --- 單一綜合提示模板 ---
COMBINED_TEMPLATE = """
您是一位金融價差分析師，使用 Neo4j 圖形資料庫分析股票 '0050.TW' 和 '2330.TW' 的價差 (PriceDiff) 與相關指標 (EMA5, EMA10, ATR, Return) 的 sign 值之間的關係。任務是：
1. 生成 Cypher 查詢，從資料庫提取數據。
2. 基於提取的數據，計算 PriceDiff_Change (使用 price_diff.diff()，即價差的逐日變化)。
3. 篩選 PriceDiff_Change > 0 的日子（價差擴大日），統計每個 sign 指標的出現情況：+1 的次數/比例、-1 的次數/比例、0 的次數/比例。
4. 計算每個 sign 指標與 PriceDiff_Change 的相關係數 (corr())，以找出影響（正相關表示該 sign +1 時價差易擴大）。
5. 基於統計和相關性，結論哪些指標最影響價差擴大（例如，EMA5_0050_sign 的正相關性高，則其 +1 是擴大因子）。

資料庫 Schema：
{schema}

指示：
- 僅使用 schema 中的節點 (Time, Stock, Kline, EMA5, EMA10, ATR, Return, PriceDiff) 和關係 (HAS_EMA5, HAS_EMA10, HAS_ATR, HAS_RETURN, AT_TIME, BETWEEN 等)。
- Cypher 查詢必須返回：t.date AS Date, e1.sign AS EMA5_0050_sign, e2.sign AS EMA5_2330_sign, ... (涵蓋所有指標的 sign for both stocks), pd.value AS PriceDiff，按 Date 排序。
- 示例 Cypher：
  MATCH (s1:Stock {{symbol: '0050.TW'}})-[:HAS_EMA5]->(e1:EMA5)-[:AT_TIME]->(t:Time),
        (s2:Stock {{symbol: '2330.TW'}})-[:HAS_EMA5]->(e2:EMA5)-[:AT_TIME]->(t),
        (pd:PriceDiff)-[:BETWEEN]->(s1), (pd)-[:BETWEEN]->(s2), (pd)-[:AT_TIME]->(t)
  RETURN t.date AS Date, e1.sign AS EMA5_0050_sign, e2.sign AS EMA5_2330_sign, pd.value AS PriceDiff
  ORDER BY t.date;

問題：{question}

步驟式思考：
1. 生成 Cypher 查詢並解釋。
2. 假設/使用提供的數據：{data}（如果無數據，生成模擬數據進行示範）。
3. 用 Python-like 邏輯計算：
   - df = pd.DataFrame(data)  # 假設數據為 DataFrame，欄位如 Date, EMA5_0050_sign, ..., PriceDiff
   - df['PriceDiff_Change'] = df['PriceDiff'].diff()  # 計算價差變化
   - expansion_days = df[df['PriceDiff_Change'] > 0]  # 篩選擴大日
   - 對於每個 sign 欄位 (e.g., 'EMA5_0050_sign')：
     - counts = expansion_days['EMA5_0050_sign'].value_counts()  # +1: 次數, -1: 次數, 0: 次數
     - total = len(expansion_days)
     - proportions = counts / total * 100  # 比例 %
     - corr = expansion_days['EMA5_0050_sign'].corr(expansion_days['PriceDiff_Change'])  # 相關係數
   - 重複以上對所有 sign 欄位。
4. 輸出 Markdown 格式：
   - **數據摘要表格**：顯示前 5 行數據 + PriceDiff_Change。
   - **統計表格**：每個 sign 的 +1/-1/0 次數、比例、與 PriceDiff_Change 的 corr。
   - **相關性結論**：列出 corr > 0.3 的指標作為正影響因子，corr < -0.3 作為負影響；描述趨勢 (e.g., "EMA5_2330_sign +1 出現 70%，corr=0.45，表示其上升強烈驅動價差擴大")。
   - **建議**：監測高 corr 指標的 sign 變化，當組合出現時考慮套利 (e.g., 若 EMA5_0050_sign = -1 且 Return_2330_sign = +1，賣 2330.TW 買 0050.TW)。

僅輸出最終分析，無多餘解釋。
"""
combined_prompt = PromptTemplate.from_template(COMBINED_TEMPLATE)

# --- 簡化的 Cypher 生成模板 ---
CYPHER_GEN_TEMPLATE = """
任務：生成 Cypher 語句來查詢 Neo4j 圖形資料庫。
指示：
- 僅使用 schema 中提供的節點 (Time, Stock, EMA5, EMA10, ATR, Return, PriceDiff) 和關係 (HAS_EMA5, HAS_EMA10, HAS_ATR, HAS_RETURN, AT_TIME, BETWEEN)。
- 查詢兩個股票 ('0050.TW' 和 '2330.TW') 的 EMA5, EMA10, ATR, Return 的 sign 值，以及 PriceDiff，按日期排序。
- 返回欄位：t.date AS Date, EMA5/EMA10/ATR/Return 的 sign 值 (e.g., EMA5_0050_sign, EMA5_2330_sign), pd.value AS PriceDiff。

Schema：
{schema}

問題：{question}

示例 Cypher：
MATCH (s1:Stock {{symbol: '0050.TW'}})-[:HAS_EMA5]->(e1:EMA5)-[:AT_TIME]->(t:Time),
      (s2:Stock {{symbol: '2330.TW'}})-[:HAS_EMA5]->(e2:EMA5)-[:AT_TIME]->(t),
      (pd:PriceDiff)-[:BETWEEN]->(s1), (pd)-[:BETWEEN]->(s2), (pd)-[:AT_TIME]->(t)
RETURN t.date AS Date, e1.sign AS EMA5_0050_sign, e2.sign AS EMA5_2330_sign, pd.value AS PriceDiff
ORDER BY t.date;

僅輸出 Cypher 查詢語句，無其他文字。
"""
cypher_gen_prompt = PromptTemplate.from_template(CYPHER_GEN_TEMPLATE)

# --- 執行範例查詢 ---
if __name__ == "__main__":
    # 驗證 Neo4j 連線
    try:
        driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
        with driver.session() as session:
            result = session.run("RETURN 'Neo4j connected!' AS message")
            print(result.single()["message"])
        driver.close()
    except Exception as e:
        print(f"Neo4j connection failed: {e}")
        exit()

    question = "分析 0050.TW 和 2330.TW 的價差price_diff發散與收斂的情況, 使用知識圖譜內其餘資料計算價差擴大的時候, 那些數值是正數那些數值是負數, 以每個時間為單位。"
    
    try:
        # 第一步：生成 Cypher 查詢
        cypher_input = cypher_gen_prompt.format(schema=graph.schema, question=question)
        print("\nCypher Prompt Input:")
        print(cypher_input)  # Debug: 檢查輸入 prompt

        cypher_query = generator_llm.invoke(cypher_input).strip()
        print("\n生成的 Cypher 查詢：")
        print(cypher_query)

        # 如果 Cypher 查詢為空，使用 fallback 查詢
        if not cypher_query or cypher_query == "":
            print("LLM 未生成有效 Cypher 查詢，使用 fallback 查詢。")
            cypher_query = """
            MATCH (s1:Stock {symbol: '0050.TW'})-[:HAS_EMA5]->(e1:EMA5)-[:AT_TIME]->(t:Time),
                  (s1)-[:HAS_EMA10]->(e3:EMA10)-[:AT_TIME]->(t),
                  (s1)-[:HAS_ATR]->(a1:ATR)-[:AT_TIME]->(t),
                  (s1)-[:HAS_RETURN]->(r1:Return)-[:AT_TIME]->(t),
                  (s2:Stock {symbol: '2330.TW'})-[:HAS_EMA5]->(e2:EMA5)-[:AT_TIME]->(t),
                  (s2)-[:HAS_EMA10]->(e4:EMA10)-[:AT_TIME]->(t),
                  (s2)-[:HAS_ATR]->(a2:ATR)-[:AT_TIME]->(t),
                  (s2)-[:HAS_RETURN]->(r2:Return)-[:AT_TIME]->(t),
                  (pd:PriceDiff)-[:BETWEEN]->(s1), (pd)-[:BETWEEN]->(s2), (pd)-[:AT_TIME]->(t)
            RETURN t.date AS Date,
                   e1.sign AS EMA5_0050_sign, e3.sign AS EMA10_0050_sign,
                   a1.sign AS ATR_0050_sign, r1.sign AS Return_0050_sign,
                   e2.sign AS EMA5_2330_sign, e4.sign AS EMA10_2330_sign,
                   a2.sign AS ATR_2330_sign, r2.sign AS Return_2330_sign,
                   pd.value AS PriceDiff
            ORDER BY t.date
            """

        # 第二步：執行 Cypher 查詢獲取數據
        try:
            data = graph.query(cypher_query)
            print("\n從 Neo4j 檢索的數據：")
            print(data)
        except Exception as e:
            print(f"執行 Cypher 查詢失敗: {e}")
            print("請檢查 Cypher 語句或資料庫結構。")
            exit()

        # 第三步：使用 Pandas 進行統計分析
        if data:
            df = pd.DataFrame(data)
            df = df.sort_values('Date')  # 確保按日期排序
            df['PriceDiff_Change'] = df['PriceDiff'].diff()
            expansion_days = df[df['PriceDiff_Change'] > 0]

            # 計算統計
            sign_columns = [col for col in df.columns if '_sign' in col]
            stats = {}
            for col in sign_columns:
                counts = expansion_days[col].value_counts()
                total = len(expansion_days)
                proportions = (counts / total * 100).round(2) if total > 0 else {}
                corr = expansion_days[col].corr(expansion_days['PriceDiff_Change']) if total > 0 else None
                stats[col] = {
                    'counts': counts.to_dict(),
                    'proportions': proportions.to_dict(),
                    'corr': round(corr, 4) if corr is not None else None
                }

            print("\nPython 計算的統計：")
            print(json.dumps(stats, indent=2))

            # 第四步：將數據和統計結果傳給 LLM 進行最終分析
            data_json = json.dumps(data)
            combined_input = combined_prompt.format(schema=graph.schema, question=question, data=data_json)
            response = generator_llm.invoke(combined_input)

            print("\nLLM 分析結果：")
            print(response)
        else:
            print("無數據返回，請檢查資料庫內容或 Cypher 查詢。")

    except Exception as e:
        print(f"An error occurred: {e}")


Neo4j Schema:
Node properties:
Time {date: STRING}
Stock {name: STRING}
Kline {low: FLOAT, open: FLOAT, high: FLOAT, date: STRING, stock: STRING, close: FLOAT, volume: FLOAT}
EMA5 {stock: STRING, value: FLOAT, date: STRING}
EMA10 {stock: STRING, value: FLOAT, date: STRING}
ATR {stock: STRING, value: FLOAT, date: STRING}
Return {stock: STRING, value: FLOAT, date: STRING}
PriceDiff {stock2: STRING, date: STRING, value: FLOAT, stock1: STRING}
Relationship properties:

The relationships:
(:Time)-[:HAS_KLINE]->(:Kline)
(:Stock)-[:STOCK_AT_TIME]->(:Time)
(:Kline)-[:HAS_EMA5]->(:EMA5)
(:Kline)-[:HAS_EMA10]->(:EMA10)
(:Kline)-[:HAS_ATR]->(:ATR)
(:Kline)-[:HAS_RETURN]->(:Return)
(:PriceDiff)-[:FROM_KLINE]->(:Kline)
Neo4j connected!

Cypher Prompt Input:

任務：生成 Cypher 語句來查詢 Neo4j 圖形資料庫。
指示：
- 僅使用 schema 中提供的節點 (Time, Stock, EMA5, EMA10, ATR, Return, PriceDiff) 和關係 (HAS_EMA5, HAS_EMA10, HAS_ATR, HAS_RETURN, AT_TIME, BETWEEN)。
- 查詢兩個股票 ('0050.TW' 和 '2330.TW') 的 EMA5, EMA10, ATR, Return 的 sign 值，以

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: sign)} {position: line: 12, column: 50, offset: 749} for query: "\n            MATCH (s1:Stock {symbol: '0050.TW'})-[:HAS_EMA5]->(e1:EMA5)-[:AT_TIME]->(t:Time),\n                  (s1)-[:HAS_EMA10]->(e3:EMA10)-[:AT_TIME]->(t),\n                  (s1)-[:HAS_ATR]->(a1:ATR)-[:AT_TIME]->(t),\n                  (s1)-[:HAS_RETURN]->(r1:Return)-[:AT_TIME]->(t),\n                  (s2:Stock {symbol: '2330.TW'})-[:HAS_EMA5]->(e2:EMA5)-[:AT_TIME]->(t),\n                  (s2)-[:HAS_EMA10]->(e4:EMA10)-[:AT_TIME]->(t),\n                  (s2)-[:HAS_ATR]->(a2:ATR)-[


從 Neo4j 檢索的數據：
[]
無數據返回，請檢查資料庫內容或 Cypher 查詢。
